In [2]:
import pandas as pd
import re

In [3]:
scheduler_file_loc = "../data/scheduler_transition.csv"
worker_xfer_loc = "../data/worker_transfer.csv"
worker_transition_loc = "../data/worker_transition.csv"
worker_loc = "../data/worker.csv"

In [4]:
w_x = pd.read_csv(worker_xfer_loc, dtype={'key': 'string', 'who': 'string'})
w_t = pd.read_csv(worker_transition_loc, dtype={'called_from':'string'})
w_i = pd.read_csv(worker_loc, dtype={'worker':'string'})

In [99]:
def confirm_workers(w_t, w_i) :
    worker_ids = w_i['worker'].unique()
    worker_t_ids = w_t['called_from'].unique()

    for id in worker_ids :
        if id not in worker_t_ids :
            return False

    for id in worker_t_ids :
        if id not in worker_ids :
            return False
    
    return True

class worker :
    id: str
    init_time: int

    ip: str
    port: str

    def __init__(self, id_inp: str, init_time_inp: int) -> None:
        self.id = id_inp
        self.init_time = init_time_inp

        _, self.ip, self.port = self.id.split(":")
        self.ip = "tcp:" + self.ip

def create_worker_objs(pdf) :
    workers = []
    for row in range(0, pdf.shape[0]) :
        workers.append(worker(pdf["worker"][row], pdf["time"][row]))
    
    return workers


In [139]:
class task :
    id: str

    task_type: str
    addtl_tuple: tuple # Unkown what these are rn
    addtl_int: int  # Unknown what these are rn

    def __init__(self, key) -> None:
        key = eval(key)
        fullname = list(key.keys())[0]
        #print(fullname)
        
        if re.search("-", fullname[0]) :
            self.task_type, self.id = fullname[0].rsplit("-",1)
        else :
            self.task_type = "NA"
            self.id = fullname[0]
        self.addtl_tuple = fullname[1:]
        self.addtl_int = list(key.values())[0]
        
        #print(self.task_type)
        #print(self.id)
        #print(self.addtl_tuple)
        #print(self.addtl_int)
    
def create_task_objs(pdf) :
    #return task(pdf["keys"][6974])
    #return task(pdf["keys"][0])
    #return task(pdf["keys"][6977])
    tasks = []
    for r in range(0, pdf.shape[0]) :
        tasks.append(task(pdf["keys"][r]))

In [140]:
w_x = pd.read_csv("../data/worker_transfer.csv")
tasks = create_task_objs(w_x)

In [100]:
if confirm_workers(w_t, w_i) :
    print("Creating workers.")
    workers = create_worker_objs(w_i)
    print(workers)
    print(workers[0].ip)
    print(workers[0].port)
else :
    print("Error creating workers: names not consistent")


Creating workers.
[<__main__.worker object at 0x7fbfc8ecb790>, <__main__.worker object at 0x7fbfc81e4220>, <__main__.worker object at 0x7fbfc80bc310>, <__main__.worker object at 0x7fbfc931db20>, <__main__.worker object at 0x7fbfc93bee50>, <__main__.worker object at 0x7fbfc93be100>, <__main__.worker object at 0x7fbfc93c7160>, <__main__.worker object at 0x7fbfc8ef5490>]
tcp://10.201.0.212
38553


In [42]:
def wxfr_get_types(pdf) :
    return pdf['type'].unique()

def wxfr_get_froms(pdf) :
    return pdf['called_from'].unique()

def wxfer_get_whos(pdf) :
    # NOTE : 'who' field can be NA?
    return pdf['who'].unique()

In [64]:
w_i

,Unnamed: 0,worker,action,stimulus_id,time
0,0,tcp://10.201.0.212:38553,add,NaN,1.713456e+09
1,1,tcp://10.201.0.212:34699,add,NaN,1.713456e+09
2,2,tcp://10.201.0.212:45449,add,NaN,1.713456e+09
3,3,tcp://10.201.0.212:38577,add,NaN,1.713456e+09
4,4,tcp://10.201.0.229:37485,add,NaN,1.713456e+09
5,5,tcp://10.201.0.229:41793,add,NaN,1.713456e+09
6,6,tcp://10.201.0.229:44955,add,NaN,1.713456e+09
7,7,tcp://10.201.0.229:45945,add,NaN,1.713456e+09


In [51]:
(w_x['bandwidth'] * w_x['duration'] == w_x['total']).value_counts()

False    5600
True     1378
Name: count, dtype: int64

In [4]:
w_x

,Unnamed: 0,start,stop,middle,duration,keys,total,bandwidth,who,type,called_from,time,compressed
0,0,1.713456e+09,1.713456e+09,1.713456e+09,0.025946,"{('array-8838909ee9756e34565341881e2e6f0c', 12...",336,1.294978e+04,tcp://10.201.0.212:45449,incoming_transfer,tcp://10.201.0.212:38577,1.713456e+09,NaN
1,1,1.713456e+09,1.713456e+09,1.713456e+09,0.025946,"{('array-8838909ee9756e34565341881e2e6f0c', 12...",336,1.294978e+04,tcp://10.201.0.212:45449,incoming_transfer,tcp://10.201.0.212:38577,1.713456e+09,NaN
2,2,1.713456e+09,1.713456e+09,1.713456e+09,0.030664,"{('array-8838909ee9756e34565341881e2e6f0c', 11...",1344,4.382926e+04,tcp://10.201.0.212:45449,incoming_transfer,tcp://10.201.0.212:38577,1.713456e+09,NaN
3,3,1.713456e+09,1.713456e+09,1.713456e+09,0.029351,"{('array-8838909ee9756e34565341881e2e6f0c', 11...",1344,4.579024e+04,tcp://10.201.0.212:38577,outgoing_transfer,tcp://10.201.0.212:45449,1.713456e+09,2533.0
4,4,1.713456e+09,1.713456e+09,1.713456e+09,0.039901,"{('array-8838909ee9756e34565341881e2e6f0c', 87...",336,8.420737e+03,tcp://10.201.0.212:38553,incoming_transfer,tcp://10.201.0.212:34699,1.713456e+09,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6973,6973,1.713456e+09,1.713456e+09,1.713456e+09,0.002767,"{('stack-b4b92b70b53be1d3b50e36696c31dac6', 36...",4194304,1.515653e+09,tcp://10.201.0.212:34699,outgoing_transfer,tcp://10.201.0.229:37485,1.713456e+09,4194662.0
6974,6974,1.713456e+09,1.713456e+09,1.713456e+09,0.055219,{('block-info-save_file-3bc2c82f3124827ed7e761...,3596,6.512203e+04,tcp://10.201.0.229:44955,outgoing_transfer,tcp://10.201.0.212:38577,1.713456e+09,679.0
6975,6975,1.713456e+09,1.713456e+09,1.713456e+09,0.742745,"{('stack-b4b92b70b53be1d3b50e36696c31dac6', 62...",4194304,5.647033e+06,tcp://10.201.0.212:38577,outgoing_transfer,tcp://10.201.0.229:44955,1.713456e+09,4194662.0
6976,6976,1.713456e+09,1.713456e+09,1.713456e+09,0.059816,"{('stack-b4b92b70b53be1d3b50e36696c31dac6', 11...",4194304,7.012052e+07,tcp://10.201.0.229:45945,outgoing_transfer,tcp://10.201.0.212:38553,1.713456e+09,4194662.0


In [5]:
w_t

,Unnamed: 0,key,start,finish,called_from,time
0,0,('block-info-_map_read_frame-c50155175b03e4c1e...,released,waiting,tcp://10.201.0.212:34699,1.713456e+09
1,1,('block-info-_map_read_frame-c50155175b03e4c1e...,waiting,ready,tcp://10.201.0.212:34699,1.713456e+09
2,2,('block-info-_map_read_frame-c50155175b03e4c1e...,ready,executing,tcp://10.201.0.212:34699,1.713456e+09
3,3,"('array-8838909ee9756e34565341881e2e6f0c', 125)",released,waiting,tcp://10.201.0.212:38553,1.713456e+09
4,4,('block-info-_map_read_frame-c50155175b03e4c1e...,released,waiting,tcp://10.201.0.212:38577,1.713456e+09
...,...,...,...,...,...,...
64988,64988,"('save_file-3bc2c82f3124827ed7e7612a1ee15778',...",memory,released,tcp://10.201.0.229:44955,1.713456e+09
64989,64989,"('save_file-3bc2c82f3124827ed7e7612a1ee15778',...",memory,released,tcp://10.201.0.229:37485,1.713456e+09
64990,64990,"('save_file-3bc2c82f3124827ed7e7612a1ee15778',...",memory,released,tcp://10.201.0.212:38577,1.713456e+09
64991,64991,"('save_file-3bc2c82f3124827ed7e7612a1ee15778',...",memory,released,tcp://10.201.0.212:45449,1.713456e+09


In [128]:
print(w_x["keys"][6974])

{('block-info-save_file-3bc2c82f3124827ed7e7612a1ee15778', 131, 0, 0): 3596}


In [3]:
sched = pd.read_csv(scheduler_file_loc, dtype={'key': 'string'})

In [4]:
sched

,Unnamed: 0,key,thread,worker,prefix,group,start,finish,stimulus_id,called_from,begins,ends,duration,size,time
0,0,('block-info-_map_read_frame-c50155175b03e4c1e...,NaN,NaN,block-info,block-info-_map_read_frame-c50155175b03e4c1ec9...,released,waiting,update-graph-1713455680.1067564,tcp://10.201.0.220:8786,NaN,NaN,NaN,NaN,1.713456e+09
1,1,('block-info-_map_read_frame-c50155175b03e4c1e...,NaN,NaN,block-info,block-info-_map_read_frame-c50155175b03e4c1ec9...,waiting,processing,update-graph-1713455680.1067564,tcp://10.201.0.220:8786,NaN,NaN,NaN,NaN,1.713456e+09
2,2,"('array-8838909ee9756e34565341881e2e6f0c', 127)",NaN,NaN,array,array-8838909ee9756e34565341881e2e6f0c,released,waiting,update-graph-1713455680.1067564,tcp://10.201.0.220:8786,NaN,NaN,NaN,NaN,1.713456e+09
3,3,"('array-8838909ee9756e34565341881e2e6f0c', 127)",NaN,NaN,array,array-8838909ee9756e34565341881e2e6f0c,waiting,processing,update-graph-1713455680.1067564,tcp://10.201.0.220:8786,NaN,NaN,NaN,NaN,1.713456e+09
4,4,('_map_read_frame-c50155175b03e4c1ec9664317841...,NaN,NaN,map_read_frame,_map_read_frame-c50155175b03e4c1ec9664317841e345,released,waiting,update-graph-1713455680.1067564,tcp://10.201.0.220:8786,NaN,NaN,NaN,NaN,1.713456e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46391,46391,"('chunk_min-5a09aab47a69898004101a2c5b6e1b96',...",NaN,NaN,chunk_min,chunk_min-5a09aab47a69898004101a2c5b6e1b96,released,forgotten,client-releases-keys-1713455876.421544,tcp://10.201.0.220:8786,NaN,NaN,NaN,NaN,1.713456e+09
46392,46392,('_map_read_frame-c50155175b03e4c1ec9664317841...,NaN,NaN,map_read_frame,_map_read_frame-c50155175b03e4c1ec9664317841e345,released,forgotten,client-releases-keys-1713455876.421544,tcp://10.201.0.220:8786,NaN,NaN,NaN,NaN,1.713456e+09
46393,46393,('block-info-_map_read_frame-c50155175b03e4c1e...,NaN,NaN,block-info,block-info-_map_read_frame-c50155175b03e4c1ec9...,released,forgotten,client-releases-keys-1713455876.421544,tcp://10.201.0.220:8786,NaN,NaN,NaN,NaN,1.713456e+09
46394,46394,"('array-8838909ee9756e34565341881e2e6f0c', 66)",NaN,NaN,array,array-8838909ee9756e34565341881e2e6f0c,released,forgotten,client-releases-keys-1713455876.421544,tcp://10.201.0.220:8786,NaN,NaN,NaN,NaN,1.713456e+09


Possible formatting?

```json
entity = {
    "name" : "tcp://....",
    "type" : "scheduler",
    "events" : {
        "key" : "('array-8838909ee9756e34565341881e2e6f0c', 127)", // break into parts
        "prefix" : "array",
        "group" : "array-8838909ee9756e34565341881e2e6f0c",
        "start" : "released", // replace start & finish with "transition type"?
        "finish" : "waiting",
        "stimulus_id" : "update-graph-1713455680.1067564",
        ...
    }
}
```

Goal is to be able to link events in the scheduler to events in workers. When the transition between 'waiting' and 'processing' occurs, that means it's being assigned a worker. That being said, it doesn't appear to know what worker did it until it goes from 'processing' to 'memory' -- aka a worker has _finished_ processing it. This is probably because, as Amal said, a worker can "steal" jobs from other workers when it's free, so the scheduler doesn't particularly care who has it, just who gives it a result.


Other possibility:
```json
entity = {
    "name" : "tcp://...",
    "type" : "scheduler",
    "tasks" : [
        {
            "name" : "('array-8838909ee9756e34565341881e2e6f0c', 127)",
            "type" : "array",
            "status" : "completed", // if "released" -> "forgotten"?
            "steps" : [
                {
                    "name" : "...",
                    "start" : "released",
                    "finish" : "waiting",
                    "time" : "1.713456e+09"
                },
                {
                    "name" : "...",
                    "start" : "waiting",
                    "finish" : "processing", 
                    "time" : "1.713456e+09"
                }
            ],
            "worker" : "..." // id of worker that completed it, if applicable
        },
    ]
}
```

In [5]:
unique_events = sched['key'].unique()
unique_events

<StringArray>
[          "('block-info-_map_read_frame-c50155175b03e4c1ec9664317841e345', 127, 0, 0, 0)",
                                         "('array-8838909ee9756e34565341881e2e6f0c', 127)",
                      "('_map_read_frame-c50155175b03e4c1ec9664317841e345', 127, 0, 0, 0)",
                            "('chunk_min-5a09aab47a69898004101a2c5b6e1b96', 127, 0, 0, 0)",
           "('block-info-_map_read_frame-c50155175b03e4c1ec9664317841e345', 126, 0, 0, 0)",
                                         "('array-8838909ee9756e34565341881e2e6f0c', 126)",
                      "('_map_read_frame-c50155175b03e4c1ec9664317841e345', 126, 0, 0, 0)",
                            "('chunk_max-f51ddbdd2414581636b6798cd76ef954', 127, 0, 0, 0)",
                            "('chunk_max-f51ddbdd2414581636b6798cd76ef954', 126, 0, 0, 0)",
                     "('chunk_max-partial-80f8e741f300d2a084fb2c4c7220e333', 63, 0, 0, 0)",
 ...
                    "('block-info-save_file-3bc2c82f3124827ed

In [6]:
ex = ["('block-info-save_file-331f10de8fa1ebe88ba72697a5c718c8', 99, 0, 0, 0)",
       "('save_file-331f10de8fa1ebe88ba72697a5c718c8', 99, 0, 0, 0)"]

sched[(sched['key'] == ex[0]) | (sched['key'] == ex[1])]

,Unnamed: 0,key,thread,worker,prefix,group,start,finish,stimulus_id,called_from,begins,ends,duration,size,time
2189,2189,('block-info-save_file-331f10de8fa1ebe88ba7269...,NaN,NaN,block-info,block-info-save_file-331f10de8fa1ebe88ba72697a...,released,waiting,update-graph-1713455680.1067564,tcp://10.201.0.220:8786,NaN,NaN,NaN,NaN,1.713456e+09
2190,2190,('block-info-save_file-331f10de8fa1ebe88ba7269...,NaN,NaN,block-info,block-info-save_file-331f10de8fa1ebe88ba72697a...,waiting,waiting,update-graph-1713455680.1067564,tcp://10.201.0.220:8786,NaN,NaN,NaN,NaN,1.713456e+09
2191,2191,('block-info-save_file-331f10de8fa1ebe88ba7269...,NaN,NaN,block-info,block-info-save_file-331f10de8fa1ebe88ba72697a...,waiting,queued,update-graph-1713455680.1067564,tcp://10.201.0.220:8786,NaN,NaN,NaN,NaN,1.713456e+09
2192,2192,"('save_file-331f10de8fa1ebe88ba72697a5c718c8',...",NaN,NaN,save_file,save_file-331f10de8fa1ebe88ba72697a5c718c8,released,waiting,update-graph-1713455680.1067564,tcp://10.201.0.220:8786,NaN,NaN,NaN,NaN,1.713456e+09
5075,5075,('block-info-save_file-331f10de8fa1ebe88ba7269...,NaN,NaN,block-info,block-info-save_file-331f10de8fa1ebe88ba72697a...,queued,processing,task-finished-1713455692.6430042,tcp://10.201.0.220:8786,NaN,NaN,NaN,NaN,1.713456e+09
5194,5194,('block-info-save_file-331f10de8fa1ebe88ba7269...,2.297068e+13,tcp://10.201.0.229:45945,block-info,block-info-save_file-331f10de8fa1ebe88ba72697a...,processing,memory,task-finished-1713455693.1540287,tcp://10.201.0.220:8786,1.713456e+09,1.713456e+09,0.000147,4056.0,1.713456e+09
5590,5590,"('save_file-331f10de8fa1ebe88ba72697a5c718c8',...",NaN,NaN,save_file,save_file-331f10de8fa1ebe88ba72697a5c718c8,waiting,processing,task-finished-1713455694.3521287,tcp://10.201.0.220:8786,NaN,NaN,NaN,NaN,1.713456e+09
6008,6008,"('save_file-331f10de8fa1ebe88ba72697a5c718c8',...",2.297068e+13,tcp://10.201.0.229:45945,save_file,save_file-331f10de8fa1ebe88ba72697a5c718c8,processing,memory,task-finished-1713455707.110486,tcp://10.201.0.220:8786,1.713456e+09,1.713456e+09,0.056904,71198016.0,1.713456e+09
6010,6010,('block-info-save_file-331f10de8fa1ebe88ba7269...,NaN,NaN,block-info,block-info-save_file-331f10de8fa1ebe88ba72697a...,memory,released,task-finished-1713455707.110486,tcp://10.201.0.220:8786,NaN,NaN,NaN,NaN,1.713456e+09
6062,6062,"('save_file-331f10de8fa1ebe88ba72697a5c718c8',...",NaN,NaN,save_file,save_file-331f10de8fa1ebe88ba72697a5c718c8,memory,forgotten,client-releases-keys-1713455721.907886,tcp://10.201.0.220:8786,NaN,NaN,NaN,NaN,1.713456e+09


Interesting minor note:

there appears to always be 2 tasks (at least in this case) : `block-info-save_file` and `save_file`. `block-info-save_file` takes approximately 0.0001 time while `save_file` takes 0.057. My guess is that `block-info-save_file` is literally setting up the run for `save_file`? 

Do multiple things to test this:
1. Ensure that `block-info` is always orders of magnitude smaller than the associated _actual_ task.
2. Actually look at and understand the python code to figure out if the cause of `block-info` is evident.
3. Ensure that `block-info` always enters `memory` before the associated task enters `processing`.

In [16]:
unique_events.tolist()
no_blockinfo = [x.replace("block-info-", "") for x in unique_events]
no_blockinfo = list(set(no_blockinfo))
no_blockinfo

['finalize-9c4f3189-0b22-43a6-a3a3-f9691a3bb046',
 'finalize-a2cf145c-4188-4644-8ffc-765e84d89f2f',
 "('getitem-93a200183209dacd31a8d37cc80736e4', 0, 0)",
 'finalize-0e1f41f1-2f63-4b4e-b7c2-53a36cca9a8b',
 'label-26c0401c-49a5-45b7-bfd1-edff7fe2a2bb',
 "('chunk_max-getitem-gt-max-aggregate-stack-b4b92b70b53be1d3b50e36696c31dac6', 130, 0, 0)",
 'finalize-6836b0d9-fbcd-49f4-a848-935965a81536',
 "('chunk_min-5a09aab47a69898004101a2c5b6e1b96', 99, 0, 0, 0)",
 "('chunk_min-5a09aab47a69898004101a2c5b6e1b96', 106, 0, 0, 0)",
 "('getitem-7cf8ae4a4b776bb3b1623bb22bb557c1', 0, 0)",
 'finalize-a0aabe54-fe4b-4079-b516-20bda26835af',
 "('chunk_min-5a09aab47a69898004101a2c5b6e1b96', 20, 0, 0, 0)",
 "('chunk_max-f51ddbdd2414581636b6798cd76ef954', 38, 0, 0, 0)",
 'chunk_max-finalize-gt-max-aggregate-label-28985a47-06c2-416e-ac8a-31a06ac973fd',
 "('getitem-stack-4efd20bcbb47f0bf36ff5b03a7b8445a', 127, 0, 0)",
 'finalize-5e71fddc-cb2c-4574-bf09-2acb7c0c6e22',
 'add-from-value-getitem-int32-finalize-da31